# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">Predict Car brand ! 🚗</p>

![](https://cdn.luxe.digital/media/2020/12/15110747/fastest-cars-world-2021-luxe-digital%402x.jpg)

img url : https://luxe.digital/lifestyle/cars/fastest-cars/

This car, so awesome...

### I will try to predict the car brand using CNN model in keras.
### [Last work](https://www.kaggle.com/jeongbinpark/predict-the-car-brand-using-cnn), I fail to predict the car brand. 
### The last CNN model had 0.33 accuracy that to predict the 3 brand. 
### It's like only predict one brand.
### I think it occured that had little data. So, I will try predict the car brand with more data
---

# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> Import Libraries and Load dataset</p>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2

import keras
from keras import models, layers
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

# This function is show max value briefly
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [ ]:
images = glob.glob('../input/the-car-connection-picture-dataset/*.jpg')

data = pd.DataFrame(images, columns=['src'])

data['brand'] = data['src'].apply(lambda x : x.split('_')[0].split('/')[-1])

data['brand'].value_counts().head(3)

### This dataset have so many data, I tried to use all data, but It had some trouble that excess memory. 
### So, I will use only 3 brand Chevrolet, Toyota, Ford. They had most data

In [ ]:
data = data[(data['brand']=='Chevrolet') |(data['brand']=='Toyota')|(data['brand']=='Ford')].reset_index(drop=True)
X= []
y= []
for i in range(len(data)):
    src = data.loc[i,'src']
    src = cv2.imread(src, cv2.IMREAD_COLOR)
    dst = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    X.append(cv2.resize(dst, dsize=(200, 300), interpolation=cv2.INTER_AREA))
    y.append(data.loc[i,'brand'])
    
fig, axes = plt.subplots(1,10,figsize=(25,10))
for i in range(0,10):
    axes[i].imshow(X[i])

In [ ]:
y[:10]

### I use gray color img. Because in last work, I realize that the color is not useful to predict car brand.
### And If use color img, It take so many times

# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> Feature engineering</p>

In [ ]:
X = np.array(X)
X = X.astype('float32')
X = X /255.0
X = X.reshape(-1,300,200,1)
y= np.array(pd.get_dummies(pd.DataFrame(y),columns=[0]))

X.shape, y.shape

## Changed the data to make model
1. Change the list to numpy array
2. Change the type int to float
3. Change the range 0 ~ 255 to 0 ~ 1
4. Change the shape (300,200) -> (300,200,1) to make model
5. Change the y data to categorical data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.3,random_state=100)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### And split the Data, X_train, X_test, y_train, y_test by using sklearn's train_test_split func

# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> Make CNN model</p>

In [ ]:
input_shape = X_train.shape[1:]
model = models.Sequential()
model.add(layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = X_train.shape[1:]))
model.add(layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.25))


model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(layers.Dropout(0.25))


model.add(layers.Flatten())
model.add(layers.Dense(256, activation = "relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation = "softmax"))
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='rmsprop', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size = 256, epochs = 12, shuffle=True, validation_data=(X_test,y_test),verbose = 0)


### To make model, I refer [Yassine Ghouzam's Notebook](https://www.kaggle.com/yassineghouzam/introduction-to-cnn-keras-0-997-top-6). 
### It very helpful for me. Thank you Yassine Ghouzam

# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> Evaluate the model</p>

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
fig,axes = plt.subplots(1,2, figsize=(15,8))
fig.suptitle("The model 's evaluation ",fontsize=20)
axes[0].plot(history.history['loss'])
axes[0].plot(history.history['val_loss'])
axes[0].set_title('Model Loss')
axes[0].set_ylabel('Loss')
axes[0].set_xlabel('Epoch')
axes[0].legend(['Train','Test'])


axes[1].plot(history.history['accuracy'])
axes[1].plot(history.history['val_accuracy'])
axes[1].set_title('Model Accuracy')
axes[1].set_ylabel('Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].legend(['Train','Test'])
plt.show()

### This model showed about 0.76 accuracy in test data

## Test data's img

In [ ]:
fig, axes = plt.subplots(1,10,figsize=(25,10))
for i in range(0,10):
    axes[i].imshow(X_test[i])

## Predict data

In [ ]:
pred = model.predict(X_test[:10])
pred = pd.DataFrame(pred).T.style.apply(highlight_max)
pred

## Real data

In [ ]:
Real = pd.DataFrame(y_test[:10]).T

Real['brand'] =["Chevrolet",'Toyota','Ford']

Real.set_index('brand').style.apply(highlight_max)

# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> Thank you</p>

### If you have more good idea, Plz give me some comments ! 